In [20]:
import pandas as pd
import numpy as np
import scipy as sp
import altair as alt
alt.renderers.enable('notebook')
alt.data_transformers.disable_max_rows()
df_train = pd.read_csv('../data/raw/train_lofi_rowid_Nov13.csv')
df_test = pd.read_csv('../data/raw/test_lofi_rowid_Nov13.csv')

df_train.head()

row_id  well_id          GR  label
0       0        0   99.005617      0
1       1        0  105.563944      0
2       2        0  105.921065      0
3       3        0  113.513354      0
4       4        0  101.523783      0

## Figure out if the depth reference point is common, or tied to surface.
1. Looks like the reference is not globally defined, i.e. the 0 depth is at surface. :(
2. The wells have a consistent length (all have 1100 points). So maybe the geological model is flat-surfaced..?


In [2]:
df_train.groupby('well_id').size().describe()

count    4000.0
mean     1100.0
std         0.0
min      1100.0
25%      1100.0
50%      1100.0
75%      1100.0
max      1100.0
dtype: float64

Lets plot a few wells with close `well_id` together in a section

In [3]:
well_ids = df_train['well_id'].unique()[:15]
df_sub = df_train[df_train['well_id'].isin(well_ids)]

In [4]:
alt.Chart(df_sub,width=50,height=800).encode(y='row_id',x='GR',column='well_id',order='row_id').mark_point()

Look at the distribution of labels per well

In [5]:
df_labels = pd.get_dummies(df_train['label'],prefix='lab')
df_labels['well_id'] = df_train['well_id']
df_labels.groupby('well_id').sum().hist(bins=55)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f01021c15d0>,
      dtype=object)

# Prototype a few functions:

In [6]:
from sklearn.linear_model import LinearRegression
def diff_sum(x):
    res = np.diff(x).sum()
    return res
def abs_diff_sum(x):
    res = np.abs(np.diff(x)).sum()
    return res

def rolling_slope(x):
    model_ols =LinearRegression()
    idx=np.arange(x.shape[0])
    model_ols.fit(idx.reshape(-1, 1), x.reshape(-1, 1))
    res = model_ols.coef_[0]
    return res

    
def apply_rolling_functions(df,col='GR',window=10,func ={'mean':np.mean,'std':np.std,'diff_sum':diff_sum,'abs_diff_sum':abs_diff_sum,'slope':rolling_slope}):
    names =[]
    for k,v in func.items():
        series = df[col].rolling(window=window,center=True,min_periods=1).apply(v,raw=True)
        colname = f'{k}_{window}_{col}'
        df.loc[:,colname] = series.values
        names.append(colname)
    df.index = np.arange(df.shape[0])
    return df[names]
def preprocess_a_well(df_well):
    df_feats_w20 = apply_rolling_functions(df_well,window=20)
    df_feats_w60 = apply_rolling_functions(df_well,window=60)
    df_feats_w150 = apply_rolling_functions(df_well,window=150)
    df_feats = pd.concat([df_well,df_feats_w20,df_feats_w60,df_feats_w150],axis=1)
    return df_feats


## Train a simple algorithm in a Grouped KFold manner

### Preprocess data

In [7]:
import tqdm
df_well_list = []
wells = df_train['well_id'].unique().tolist()[:50]
for w in tqdm.tqdm(wells):
    df_well = df_train[df_train['well_id'] == w]
    df_new = preprocess_a_well(df_well.copy())
    df_well_list.append(df_new.copy())
    

100%|██████████| 50/50 [00:37<00:00,  1.35it/s]


In [8]:
df_train_preprocessed = pd.concat(df_well_list,axis=0)
df_train_preprocessed.index = np.arange(df_train_preprocessed.shape[0])

In [9]:
df_well_list[2]

row_id  well_id          GR  label  mean_20_GR  std_20_GR  \
0          0        2   98.146986      0  102.330481   4.130595   
1          1        2  109.258152      0  102.938364   4.382460   
2          2        2   99.504510      0  103.279497   4.345751   
3          3        2  100.374358      0  103.364987   4.185752   
4          4        2  101.262440      0  103.033227   4.207124   
...      ...      ...         ...    ...         ...        ...   
1095    1095        2  107.056298      0  102.698416   4.159607   
1096    1096        2  104.810265      0  102.703522   4.305557   
1097    1097        2  105.483216      0  102.585261   4.446123   
1098    1098        2  101.632472      0  102.837828   4.537188   
1099    1099        2  107.022714      0  103.200714   4.569168   

      diff_sum_20_GR  abs_diff_sum_20_GR  slope_20_GR  mean_60_GR  ...  \
0           2.495593           46.590760     0.029246  103.573749  ...   
1          10.870205           54.965372     0.325876  103.672051  ...   
2           8.884976           56.950602     0.408120  103.455475  ...   
3           6.243884           59.591693     0.357304  103.293945  ...   
4           0.573358           65.262219     0.153139  103.197073  ...   
...              ...                 ...          ...         ...  ...   
1095        4.395776           59.814330     0.204955  102.908477  ...   
1096        2.781800           58.200354     0.249895  102.954293  ...   
1097        7.468249           53.513904     0.371500  102.834724  ...   
1098        8.176639           52.805515     0.335054  102.983467  ...   
1099        5.678469           50.307346     0.217839  103.019175  ...   

      mean_60_GR  std_60_GR  diff_sum_60_GR  abs_diff_sum_60_GR  slope_60_GR  \
0     103.573749   4.385600        0.915140          128.285618    -0.038175   
1     103.672051   4.347752        8.474130          135.844607    -0.016165   
2     103.455475   4.445930       -1.405361          145.724099    -0.054073   
3     103.293945   4.472388       -0.021997          147.107463    -0.077807   
4     103.197073   4.441129        1.853312          148.982772    -0.087744   
...          ...        ...             ...                 ...          ...   
1095  102.908477   3.683817        5.671970          150.356844    -0.010379   
1096  102.954293   3.727758        0.122645          144.807518    -0.019652   
1097  102.834724   3.719030        8.947762          135.982401     0.000925   
1098  102.983467   3.678764        5.146190          132.180829    -0.027775   
1099  103.019175   3.732165       -0.083719          126.950920    -0.037694   

      mean_150_GR  std_150_GR  diff_sum_150_GR  abs_diff_sum_150_GR  \
0      102.510180    4.675853        -0.464817           362.542224   
1      102.496564    4.646485         3.328424           366.335465   
2      102.536529    4.629344         7.426887           370.433928   
3      102.584116    4.618489         8.101330           371.108371   
4      102.645796    4.621382         9.309833           372.316874   
...           ...         ...              ...                  ...   
1095   103.115946    4.764421         3.490542           424.489470   
1096   103.110678    4.794250         1.396810           422.395737   
1097   103.078431    4.816364         7.922464           415.870084   
1098   103.130096    4.826015         2.612485           410.560105   
1099   103.113252    4.855412        -4.597551           403.350069   

      slope_150_GR  
0        -0.039911  
1        -0.039417  
2        -0.034826  
3        -0.029890  
4        -0.024143  
...            ...  
1095     -0.012928  
1096     -0.013020  
1097     -0.011014  
1098     -0.015528  
1099     -0.014801  

[1100 rows x 34 columns]

### Train a model

In [19]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score,accuracy_score
from sklearn.dummy import DummyClassifier
X = df_train_preprocessed.drop(columns=['row_id','label','well_id'])
y = df_train_preprocessed['label']
groups = df_train_preprocessed['well_id']
group_kfold = GroupKFold(n_splits=5)

for train_index, test_index in group_kfold.split(X, y, groups):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = LGBMClassifier(class_weight='balanced',n_jobs=1)
    dumbo = DummyClassifier(strategy='most_frequent')
    dumbo.fit(X_train,y_train)
    model.fit(X_train,y_train)
    
    score = f1_score(y_test,model.predict(X_test),average='weighted')
    acc = accuracy_score(y_test,model.predict(X_test))
    
    score_dumb = f1_score(y_test,dumbo.predict(X_test),average='weighted')
    acc_dumb = accuracy_score(y_test,dumbo.predict(X_test))
    
    print(f'f1 score = {score} , accuracy = {acc}')
    print(f'Dummy f1 score = {score_dumb} , Dummy accuracy = {acc_dumb}')
    
    



/home/anton/miniconda3/envs/pg_model/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


f1 score = 0.8872695884607005 , accuracy = 0.8863636363636364
Dummy f1 score = 0.35806044678055193 , Dummy accuracy = 0.522
f1 score = 0.8857300025003316 , accuracy = 0.8859090909090909
Dummy f1 score = 0.3430751769320099 , Dummy accuracy = 0.5087272727272727
f1 score = 0.8988268575928188 , accuracy = 0.8973636363636364
Dummy f1 score = 0.35981839879793215 , Dummy accuracy = 0.5235454545454545
f1 score = 0.8920672049506971 , accuracy = 0.8907272727272727
Dummy f1 score = 0.36427702329809225 , Dummy accuracy = 0.5274545454545455
f1 score = 0.8880358721347565 , accuracy = 0.8877272727272727
Dummy f1 score = 0.37031812789819907 , Dummy accuracy = 0.5327272727272727


### Run dummy on test and prep a submission

In [22]:
import tqdm
df_well_list_test = []
wells_test = df_test['well_id'].unique().tolist()
for w in tqdm.tqdm(wells_test):
    df_well = df_test[df_test['well_id'] == w]
    df_new = preprocess_a_well(df_well.copy())
    df_well_list_test.append(df_new.copy())
df_test_preprocessed = pd.concat(df_well_list_test,axis=0)
df_test_preprocessed.index = np.arange(df_test_preprocessed.shape[0])

100%|██████████| 350/350 [04:31<00:00,  1.35it/s]


In [24]:
df_test_preprocessed.tail()

row_id  well_id          GR  mean_20_GR  std_20_GR  diff_sum_20_GR  \
384995    1095     5349  134.221769  127.554726   6.002246        2.500134   
384996    1096     5349  135.804491  127.376134   6.174292        5.557527   
384997    1097     5349  126.124399  127.405250   6.406440        6.482308   
384998    1098     5349  117.591583  127.516284   6.656005       15.765916   
384999    1099     5349  132.555150  128.491470   6.076021        7.436606   

        abs_diff_sum_20_GR  slope_20_GR  mean_60_GR  std_60_GR  ...  \
384995          100.036230     0.137695  127.065247   5.358554  ...   
384996           96.978837     0.251898  127.012159   5.427706  ...   
384997           96.054056     0.300314  127.013795   5.509322  ...   
384998           86.770447     0.321654  127.093298   5.576070  ...   
384999           78.441137    -0.166961  127.079569   5.664761  ...   

        mean_60_GR  std_60_GR  diff_sum_60_GR  abs_diff_sum_60_GR  \
384995  127.065247   5.358554        3.684930          212.445466   
384996  127.012159   5.427706        5.596954          210.533442   
384997  127.013795   5.509322        8.085466          208.044930   
384998  127.093298   5.576070        5.036239          204.995702   
384999  127.079569   5.664761        3.699783          203.659247   

        slope_60_GR  mean_150_GR  std_150_GR  diff_sum_150_GR  \
384995     0.048411   122.876934   18.397732        77.915231   
384996     0.062464   123.740694   16.824821        81.099872   
384997     0.068014   124.667430   14.793763        80.761528   
384998     0.059208   125.613843   12.322064        79.357862   
384999     0.067875   126.566693    9.160845        73.545930   

        abs_diff_sum_150_GR  slope_150_GR  
384995           479.151857      0.327365  
384996           475.967216      0.273513  
384997           475.628871      0.211957  
384998           474.225206      0.145606  
384999           468.413274      0.075203  

[5 rows x 33 columns]

In [28]:
submission_dummy = df_test_preprocessed.loc[:,['row_id','well_id']]
submission_dummy['label'] = dumbo.predict(df_test_preprocessed.drop(columns=['row_id','well_id']))
submission_dummy.to_csv('../data/final/dummy.csv',index=False)
